In [ ]:
#encoding:utf-8
###########import packages##########
import shap
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost 
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from keras.models import Model,load_model
import shap
from catboost import *
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=np.sqrt(sum(squaredError)/len(squaredError))
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('datatest.csv',encoding="gbk")
raw_data=fdata.loc[:,['Dielectric Constant (C  N-1 M-2)',#0
                      'Flash Point(℃)',#1
                      'Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)',#2
                      'Flow Rate (mL min-1)',#3
                      'Water Content (wt%)',#4
                      'Anodic Platinum Loading Amount (mgPt cm-2)',#5
                      'Cathodic Platinum Loading Amount (mgPt cm-2)',#6
                      'I/C',#7
                      'Cell Temperature (℃)',#8
                      'Anode Flow Rate (sccm)',#9
                      'Cathode Flow Rate (sccm)',#10
                      'Solid Content (wt%)',#11
                      'Backpressure (Mpa)',#18
                      'Pt Consumption per kW@0.65V (mgpt kW-1)'#19
                        ]]



###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=raw_data.iloc[:,0:13]
raw_output=raw_data.iloc[:,13]
raw_input_global=raw_data.iloc[:,0:13]
raw_output_global=raw_data.iloc[:,13]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})

In [ ]:
model_ANN=load_model('ANN_PTUTIL_1_sel.h5')

In [ ]:
from pdpbox import pdp
def plot_pdp_interact_ANN(model, df, f_list, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    inter1 = pdp.pdp_interact(model, df, model_features=df.columns.tolist(), features=f_list,num_grid_points=[20,20])
    # plot it
    settings = {
            'contour_color':  'white',
            'font_family': 'Arial',
            # matplotlib color map for interact plot
            'cmap': 'viridis',
            # fill alpha for interact plot
            'inter_fill_alpha': 0.8,
            # fontsize for interact plot text
            'inter_fontsize': 7,
        }
    pdp.pdp_interact_plot(
    pdp_interact_out=inter1, feature_names=f_list, plot_type='contour',figsize=(10,10),x_quantile=True, plot_pdp=True,plot_params=settings
)

In [ ]:
def plot_pdp_single(model, df, feature, cluster_flag=False, nb_clusters=None, lines_flag=False):
    
    # Create the data that we will plot
    pdp_goals = pdp.pdp_isolate(model=model, dataset=df, model_features=df.columns.tolist(), feature=feature)

    # plot it
    pdp.pdp_plot(pdp_goals, feature, cluster=cluster_flag, n_cluster_centers=nb_clusters, plot_lines=lines_flag)
    plt.show()
def plot_pdp_single_else(model,param,feature):
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    # Create the data that we will plot
    plot_pdp_single(best_model,raw_input,feature)

In [ ]:
inter_list1=['Dielectric Constant (C  N-1 M-2)',
                      'Flash Point(℃)']
inter_list2=['Anodic Platinum Loading Amount (mgPt cm-2)',#9
                      'Cathodic Platinum Loading Amount (mgPt cm-2)']
inter_list3=['Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)',
'Solid Content (wt%)']
inter_list4=['I/C', 'Water Content (wt%)']
# plot_pdp_interact_ANN(model_ANN,raw_input,inter_list1)
# plot_pdp_interact_ANN(model_ANN,raw_input,inter_list2)
# plot_pdp_interact_ANN(model_ANN,raw_input,inter_list3)
plot_pdp_interact_ANN(model_ANN,raw_input,inter_list4)
# plot_pdp_single(model_ANN,raw_input,'Flow Rate (mL min-1)')

In [ ]:
from sklearn.inspection import plot_partial_dependence
from sklearn.utils import validation
def pdp_plot_2d(model,param,f_index):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_    
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_index], X=raw_input, percentiles=(0, 1))
def pdp_plot_2d_XG_CAT(model,param,f_index):
    print('start')
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    best_model.dummy_ = "dummy"
    validation.check_is_fitted(estimator=best_model)
    my_plots =plot_partial_dependence(best_model, features=[f_index], X=raw_input, percentiles=(0, 1))
def pdp_plot_2d_ANN(model,f_index):
    print('start')
    model.dummy_ = "dummy"
    print(type(model))
    validation.check_is_fitted(estimator=model)
    my_plots =plot_partial_dependence(model, features=[f_index], X=raw_input, percentiles=(0, 1))

In [ ]:
def processing_PRINT_ANN(DC,FP,SM,FR,WC,AL,CL,IC,SC,model):
    raw_median=raw_data.median()
    user_input1=raw_median[0:13]    
    dict_result={}    
    user_input1.iloc[0]=DC
    user_input1.iloc[1]=FP
    user_input1.iloc[2]=SM
    user_input1.iloc[3]=FR
    user_input1.iloc[4]=WC
    user_input1.iloc[5]=AL
    user_input1.iloc[6]=CL
    user_input1.iloc[7]=IC
    user_input1.iloc[8]=80
    user_input1.iloc[9]=300
    user_input1.iloc[10]=1500
    user_input1.iloc[11]=SC
    user_input1.iloc[12]=0.2
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:13])/np.std(raw_data,axis=0)[0:13]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
#     print(type(x_prediction_ann[0]))
#     dict_result.update({x_prediction_ann[0]:{'Dielectric Constant (C  N-1 M-2)':DC,'Flash Point(℃)':FP}})
    print(DC,FP,SM,FR,WC,AL,CL,IC,SC,x_prediction_ann[0])

    return x_prediction_ann[0], dict_result

In [ ]:
def processing_PRINT_XG(DC,FP,SM,FR,WC,AL,CL,IC,SC,model):
    raw_median=raw_data.median()
    user_input1=raw_median[0:13]    
    dict_result={}    
    user_input1.iloc[0]=DC
    user_input1.iloc[1]=FP
    user_input1.iloc[2]=SM
    user_input1.iloc[3]=FR
    user_input1.iloc[4]=WC
    user_input1.iloc[5]=AL
    user_input1.iloc[6]=CL
    user_input1.iloc[7]=IC
    user_input1.iloc[8]=80
    user_input1.iloc[9]=300
    user_input1.iloc[10]=1500
    user_input1.iloc[11]=SC
    user_input1.iloc[12]=0.2
    data_test_input1=pd.DataFrame(user_input1)
    data_test_input1=data_test_input1.T
#     data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_input1)
    
    x_prediction_ann=predict_ann.astype(np.float32)
#     print(type(x_prediction_ann[0]))
#     dict_result.update({x_prediction_ann[0]:{'Dielectric Constant (C  N-1 M-2)':DC,'Flash Point(℃)':FP}})
    print(DC,FP,SM,FR,WC,AL,CL,IC,SC,x_prediction_ann[0])

    return x_prediction_ann, dict_result

In [ ]:
def processing_PRINT_OTHER(DC,FP,SM,FR,WC,AL,CL,IC,SC,model):
    raw_median=raw_data.median()
    user_input1=raw_median[0:13]    
    dict_result={}    
    user_input1.iloc[0]=DC
    user_input1.iloc[1]=FP
    user_input1.iloc[2]=SM
    user_input1.iloc[3]=FR
    user_input1.iloc[4]=WC
    user_input1.iloc[5]=AL
    user_input1.iloc[6]=CL
    user_input1.iloc[7]=IC
    user_input1.iloc[8]=80
    user_input1.iloc[9]=300
    user_input1.iloc[10]=1500
    user_input1.iloc[11]=SC
    user_input1.iloc[12]=0.2
    data_test_input1=pd.DataFrame(user_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model.predict(data_test_param1)
    
    x_prediction_ann=predict_ann.astype(np.float32)
#     print(type(x_prediction_ann[0]))
#     dict_result.update({x_prediction_ann[0]:{'Dielectric Constant (C  N-1 M-2)':DC,'Flash Point(℃)':FP}})
    print(DC,FP,SM,FR,WC,AL,CL,IC,SC,x_prediction_ann[0])

    return x_prediction_ann, dict_result

In [ ]:
def gridsearch(model,param,algorithm_name):
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    best_model=grid.best_estimator_
    result = best_model.predict(X_test)
    x_prediction_07=result
    y_real_07=y_test.values
    x_prediction_07_series=pd.Series(x_prediction_07)
    y_real_07_series=pd.Series(y_real_07)
    
    result_train = best_model.predict(X_train)
    x_prediction_07_train=result_train
    y_real_07_train=y_train.values
    x_prediction_07_series_train=pd.Series(x_prediction_07_train)
    y_real_07_series_train=pd.Series(y_real_07_train)
    
    ###########evaluating the regression quality##########
    corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
    error_val= compute_mae_mse_rmse(x_prediction_07,y_real_07)
    
    corr_ann_train = round(x_prediction_07_series_train.corr(y_real_07_series_train), 5)
    error_val_train= compute_mae_mse_rmse(x_prediction_07_train,y_real_07_train)
    
    print(algorithm_name)
    print(best_model.feature_importances_)
    print('Best Regressor:',grid.best_params_,'Best Score:', grid.best_score_)
    print(error_val,'TEST R2',error_val[3],'TEST CORR',corr_ann)
    print(error_val_train,'TRAIN R2',error_val_train[3],'TRAIN CORR',corr_ann_train)
    x_y_x=np.arange(0,2.1,0.1)
    x_y_y=np.arange(0,2.1,0.1)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.scatter(x_prediction_07,y_real_07,color='red',label=algorithm_name+' Test Set',alpha=0.75)
    ax.scatter(x_prediction_07_train,y_real_07_train,color='blue',label=algorithm_name+' Training Set',alpha=0.25,marker="^")
    ax.plot(x_y_x,x_y_y)
    plt.legend()
    plt.xlabel(u"Predicted_Pt_Consumption_per_kW@0.65V (mgpt kW-1)")
    plt.ylabel(u"Real_Pt_Consumption_per_kW@0.65V (mgpt kW-1)")
    print('finished')
    return best_model

In [ ]:
def shap_plot_interaction(model,param,algorithm_name,interacted_features):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:13]
    SHAP_OUTPUT=raw_data.iloc[:,13]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    print('train finished')
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        
#         shap_values = model.get_feature_importance(Pool(data[Xtrain.columns]),type=EFstrType.ShapValues,verbose=100)
        interaction_values = best_model.get_feature_importance(Pool(SHAP_INPUT),type=EFstrType.ShapInteractionValues)
        interaction_values=interaction_values[:,:-1,:-1]
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
        shap.summary_plot(interaction_values,SHAP_INPUT, max_display=14)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)        
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)        
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
        shap.summary_plot(interaction_values,SHAP_INPUT, max_display=14)
    else:
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)        
        shap.dependence_plot(interacted_features[0], shap_values, SHAP_INPUT,interaction_index= interacted_features[1])
        shap.dependence_plot(interacted_features[1], shap_values, SHAP_INPUT,interaction_index= interacted_features[0])
        shap.summary_plot(interaction_values, SHAP_INPUT, max_display=14)
def shap_plot_interaction_ANN(model,interacted_features):
    SHAP_INPUT=raw_data.iloc[:,0:13]
    SHAP_OUTPUT=raw_data.iloc[:,13]
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    explainer = shap.DeepExplainer(model,X_SHAP)
    shap_values = explainer.shap_values(X_SHAP)
    shap.dependence_plot(interacted_features[0], shap_values[0], SHAP_INPUT,interaction_index= interacted_features[1])
    shap.dependence_plot(interacted_features[1], shap_values[0], SHAP_INPUT,interaction_index= interacted_features[0])

In [ ]:
def shap_plot_interaction_single(model,param,algorithm_name,interacted_feature):
    print(algorithm_name)
    SHAP_INPUT=raw_data.iloc[:,0:13]
    SHAP_OUTPUT=raw_data.iloc[:,13]
    grid = GridSearchCV(model,param_grid=param,cv=5)
    grid.fit(X_train,y_train)
    print('train finished')
    best_model=grid.best_estimator_
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)

    if algorithm_name=='CatBoost':
        shap_values = best_model.get_feature_importance(Pool(X_SHAP,y_SHAP), type="ShapValues")
        shap_values=shap_values[:,:-1]
        
#         shap_values = model.get_feature_importance(Pool(data[Xtrain.columns]),type=EFstrType.ShapValues,verbose=100)
        interaction_values = best_model.get_feature_importance(Pool(SHAP_INPUT),type=EFstrType.ShapInteractionValues)
        interaction_values=interaction_values[:,:-1,:-1]
        shap.dependence_plot(interacted_feature, shap_values, SHAP_INPUT, interaction_index=None, show=False)
    elif algorithm_name=='Random Forest' or algorithm_name=='Extra Tree'or algorithm_name=='Decision Tree'or algorithm_name=='AdaBoost':
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)        
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)        
        shap.dependence_plot(interacted_feature, shap_values, SHAP_INPUT,interaction_index=None, show=False)
    else:
        explainer = shap.TreeExplainer(best_model,SHAP_INPUT)
        shap_values = explainer.shap_values(X_SHAP,check_additivity= False)
        interaction_values = shap.TreeExplainer(best_model).shap_interaction_values(SHAP_INPUT)        
        shap.dependence_plot(interacted_feature, shap_values, SHAP_INPUT, interaction_index=None, show=False)
def shap_plot_interaction_ANN_single(model,interacted_feature):
    SHAP_INPUT=raw_data.iloc[:,0:13]
    SHAP_OUTPUT=raw_data.iloc[:,13]
    X_SHAP=SHAP_INPUT.values.astype(np.float32)
    y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
    explainer = shap.DeepExplainer(model,X_SHAP)
    shap_values = explainer.shap_values(X_SHAP)
    shap.dependence_plot(interacted_feature, shap_values[0], SHAP_INPUT, interaction_index=None, show=False)

In [ ]:
shap_plot_interaction_ANN_single(model_ANN,"Flow Rate (mL min-1)")

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(15,85,5):
    for j in np.arange(-20,125,5):
        processing_PRINT_ANN(i,j,0,0.3,0.27,0.05,0.1,0.54,0.045,model_ANN)        

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0.05,0.8,0.05):
    for j in np.arange(0.05,0.8,0.05):
        processing_PRINT_ANN(18.3,12,0,0.3,0.27,i,j,0.54,0.045,model_ANN)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,2,1):
    for j in np.arange(0,0.11,0.005):
        processing_PRINT_ANN(18.3,12,i,0.3,0.27,0.05,0.1,0.54,j,model_ANN)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.05):
    for j in np.arange(0,4,0.05):
        processing_PRINT_ANN(18.3,12,0,0.3,i,0.05,0.1,j,0.045,model_ANN)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.01):
    processing_PRINT_ANN(18.3,12,0,i,0.27,0.05,0.1,0.54,0.045,model_ANN)  

In [ ]:
f_list1=['Dielectric Constant (C  N-1 M-2)','Flash Point(℃)']
f_list2=['Anodic Platinum Loading Amount (mgPt cm-2)','Cathodic Platinum Loading Amount (mgPt cm-2)']
f_list3=['Stirring Method (0 for Mechanical Stirring 1 for Ultrasound)','Solid Content (wt%)']
f_list4=['Water Content (wt%)','I/C']
f_index1=(0,1)
f_index2=(5,6)
f_index3=(2,11)
f_index4=(4,7)

In [ ]:
shap_plot_interaction_ANN(model_ANN,f_list1)

In [ ]:
shap_plot_interaction_ANN(model_ANN,f_list2)

In [ ]:
shap_plot_interaction_ANN(model_ANN,f_list3)

In [ ]:
shap_plot_interaction_ANN(model_ANN,f_list4)

In [ ]:
def processing_ANN(DC,FP,SM,FR,WC,AL,CL,IC,SC):
    raw_median=raw_data.median()
    user_input1=raw_median[0:13]    
    dict_result={}    
    user_input1.iloc[0]=DC
    user_input1.iloc[1]=FP
    user_input1.iloc[2]=SM
    user_input1.iloc[3]=FR
    user_input1.iloc[4]=WC
    user_input1.iloc[5]=AL
    user_input1.iloc[6]=CL
    user_input1.iloc[7]=IC
    user_input1.iloc[8]=80
    user_input1.iloc[9]=300
    user_input1.iloc[10]=1500
    user_input1.iloc[11]=SC
    user_input1.iloc[12]=0.2
    standardized_user_input1=(user_input1-np.mean(raw_data,axis=0)[0:13])/np.std(raw_data,axis=0)[0:13]
    data_test_input1=standardized_user_input1
    data_test_input1=pd.DataFrame(data_test_input1)
    data_test_input1=data_test_input1.T
    data_test_param1=data_test_input1.values.astype(np.float32)
    predict_ann=model_ANN.predict(data_test_param1)
    ###########result output##########
    predict_ann=predict_ann*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
    x_prediction_ann=predict_ann.astype(np.float32)
    x_prediction_ann=x_prediction_ann.tolist()
    x_prediction_ann=x_prediction_ann[0]
#     print(type(x_prediction_ann[0]))
    dict_result.update({x_prediction_ann[0]:{'Dielectric Constant (C  N-1 M-2)':DC,'Flash Point(℃)':FP}})
#     print(DC,FP,SM,FR,AL,CL,SC,x_prediction_ann[0])

    return x_prediction_ann[0], dict_result
def processing_func(p):
    DC,FP,SM,FR,WC,AL,CL,IC,SC= p
    result, _ = processing_ANN(DC,FP,SM,FR,WC,AL,CL,IC,SC)
    return result

In [ ]:
from sko.GA import GA
from sko.DE import DE
%matplotlib
iter_num = 32
ga = GA(func=processing_func, n_dim=7, size_pop=50, max_iter=iter_num, lb=[15,-20,0,0,0.05,0.1,0], ub=[80,120,1,1,0.75,0.75,0.105],precision=1e-2)
import time
time_start = time.time()
best_x, best_y = ga.run()
print('best_x:', best_x, '\n', 'best_y:', best_y)
time_end = time.time()
print('Time cost of grid search= %fs' % (time_end - time_start))

# visualization 
import pandas as pd
import matplotlib.pyplot as plt

Y_history = pd.DataFrame(ga.all_history_Y)
fig, ax = plt.subplots(2, 1, figsize=(6, 9))
ax[0].plot(Y_history.index, Y_history.values, '.', color='red')
Y_history.min(axis=1).cummin().plot(kind='line')
plt.show()
plt.savefig('GA OPT ANN SEL.png')

In [ ]:
seed=17
X_train, X_test, y_train, y_test = train_test_split(raw_input, raw_output, test_size=.15,random_state=seed)
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)
raw_input_global=raw_data.iloc[:,0:13]
raw_output_global=raw_data.iloc[:,13]

In [ ]:
##########CatBoost gridsearch CV for best hyperparameter##########
model_CatRegressor=catboost.CatBoostRegressor(random_state=1,verbose=0)
param_cat = {
 'learning_rate':[0.16],
 'n_estimators':[50],
 'max_depth':[5],
 'subsample':[0.4],
    'reg_lambda':[0.0001]
}
CAT=gridsearch(model_CatRegressor,param_cat,'CatBoost')

In [ ]:
plot_pdp_single_else(model_CatRegressor,param_cat,'Flow Rate (mL min-1)')

In [ ]:
shap_plot_interaction_single(model_CatRegressor,param_cat,'CatBoost','Flow Rate (mL min-1)')

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(15,85,5):
    for j in np.arange(-20,125,5):
        processing_PRINT_OTHER(i,j,0,0.3,0.27,0.05,0.1,0.54,0.045,CAT)        

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0.05,0.8,0.05):
    for j in np.arange(0.05,0.8,0.05):
        processing_PRINT_OTHER(18.3,12,0,0.3,0.27,i,j,0.54,0.045,CAT)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,2,1):
    for j in np.arange(0,0.11,0.005):
        processing_PRINT_OTHER(18.3,12,i,0.3,0.27,0.05,0.1,0.54,j,CAT)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.05):
    for j in np.arange(0,4,0.05):
        processing_PRINT_OTHER(18.3,12,0,0.3,i,0.05,0.1,j,0.045,CAT)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.01):
    processing_PRINT_OTHER(18.3,12,0,i,0.27,0.05,0.1,0.54,0.045,CAT)  

In [ ]:
shap_plot_interaction(model_CatRegressor,param_cat,'CatBoost',f_list1)

In [ ]:
shap_plot_interaction(model_CatRegressor,param_cat,'CatBoost',f_list2)

In [ ]:
shap_plot_interaction(model_CatRegressor,param_cat,'CatBoost',f_list3)

In [ ]:
shap_plot_interaction(model_CatRegressor,param_cat,'CatBoost',f_list4)

In [ ]:
pdp_plot_2d_XG_CAT(model_CatRegressor,param_cat,f_index1)

In [ ]:
pdp_plot_2d_XG_CAT(model_CatRegressor,param_cat,f_index2)

In [ ]:
pdp_plot_2d_XG_CAT(model_CatRegressor,param_cat,f_index3)

In [ ]:
pdp_plot_2d_XG_CAT(model_CatRegressor,param_cat,f_index4)

In [ ]:
###########GradientBoost gridsearch CV for best hyperparameter##########
model_GradientBoostingRegressor = ensemble.GradientBoostingRegressor(random_state=1)
###########defining the parameters dictionary##########
param_GB = {
 'learning_rate':[0.2],
                'criterion':['mae'],
                 'max_features':['sqrt'],
                 'loss':['ls'],
    'max_depth':[3]
}
GB=gridsearch(model_GradientBoostingRegressor,param_GB,'GradientBoost')

In [ ]:
shap_plot_interaction_single(model_GradientBoostingRegressor,param_GB,'Gradient Boost','Flow Rate (mL min-1)')

In [ ]:
plot_pdp_single_else(model_GradientBoostingRegressor,param_GB,'Flow Rate (mL min-1)')

In [ ]:
shap_plot_interaction(model_GradientBoostingRegressor,param_GB,'GradientBoost',f_list1)

In [ ]:
shap_plot_interaction(model_GradientBoostingRegressor,param_GB,'GradientBoost',f_list2)

In [ ]:
shap_plot_interaction(model_GradientBoostingRegressor,param_GB,'GradientBoost',f_list3)

In [ ]:
shap_plot_interaction(model_GradientBoostingRegressor,param_GB,'GradientBoost',f_list4)

In [ ]:
pdp_plot_2d(model_GradientBoostingRegressor,param_GB,f_index1)

In [ ]:
pdp_plot_2d(model_GradientBoostingRegressor,param_GB,f_index2)

In [ ]:
pdp_plot_2d(model_GradientBoostingRegressor,param_GB,f_index3)

In [ ]:
pdp_plot_2d(model_GradientBoostingRegressor,param_GB,f_index4)

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(15,85,5):
    for j in np.arange(-20,125,5):
        processing_PRINT_OTHER(i,j,0,0.3,0.27,0.05,0.1,0.54,0.045,GB)        

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0.05,0.8,0.05):
    for j in np.arange(0.05,0.8,0.05):
        processing_PRINT_OTHER(18.3,12,0,0.3,0.27,i,j,0.54,0.045,GB)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,2,1):
    for j in np.arange(0,0.11,0.005):
        processing_PRINT_OTHER(18.3,12,i,0.3,0.27,0.05,0.1,0.54,j,GB)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.05):
    for j in np.arange(0,4,0.05):
        processing_PRINT_OTHER(18.3,12,0,0.3,i,0.05,0.1,j,0.045,GB)  

In [ ]:
#DC,FP,SM,FR,WC,AL,CL,IC,SC,model
for i in np.arange(0,1,0.01):
    processing_PRINT_OTHER(18.3,12,0,i,0.27,0.05,0.1,0.54,0.045,GB)  